In [1]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch
from collections import defaultdict
import copy
import math

2022-09-12 02:24:57.555363: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [22]:

model = SentenceTransformer('all-mpnet-base-v2')

df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

# with open('PreProcessed_segmented.pkl', 'rb') as f:
#     newmetalist=pickle.load(f)

with open('New_PreProcessed_segmented.pkl', 'rb') as f:
    newnewmetalist=pickle.load(f)

sampled_datadf=pd.read_csv("Sampled_Adword_Labeled.csv")

#For Normal process with target labels only

df1=pd.read_csv('ChildAdwordTargetDatabase.csv')
target_dict=dict(zip(df1['Name'], df1.index))

In [23]:
def clean(text):
    text=text.lower()
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    text=text.replace("&", "")
    return text

In [ ]:
def process_text(text):
    #doc = self.nlp(unicode(text.lower()))
    text=text.replace('\\n', ' ')
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return nlp(" ".join(result))

In [4]:
df1.head()

,Unnamed: 0,Name,info
0,0,Automotive,Automotive Crossover Hatchback Microcar Off-Ro...
1,1,Books and Literature,Books and Literature Art and Photography Books...
2,2,Business and Finance,Business and Finance Business Business Account...
3,3,Education,Education College Education Primary Education ...
4,4,Events and Attractions,Events and Attractions Fashion Events Malls & ...


In [24]:

# df1['Combined']=df1['Name'] #+' '+df1['info']
# df1['Combined1']=df1['Name'] +' '+df1['info']
# df1['Combined1'].apply(clean)
# df1['Combined1'].apply(process_text)
# df1['Combined'].apply(clean)

df1['info']=df1['info'].apply(clean)
#df1['info']=df1['info'].apply(process_text)
targetlist=df1['Name'].to_list()
# targetlistcalc=df1['Combined1'].to_list()
targetlistchild=df1['info']

In [25]:
truthdatadf=pd.read_csv('AdwordLabeledDatabase.csv')
truthdatadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,3,3,GMO_00000000001313_01,FILE_MAF_20220211T205601Z_GMO_00000000001313_01,FULL HEARTS (EDITED),friday night lights,Is something happening\nbetween you and Riggin...,Sports
1,6,6,GMO_00000000001534_01,FILE_MAF_20220211T205607Z_GMO_00000000001534_01,MAY THE BEST MAN WIN (EDITED),friday night lights,BE HONEST WITH YOURSELF\nFOR TWO SECONDS AND T...,Sports
2,7,7,GMO_00000000001577_01,FILE_MAF_20210727T231031Z_GMO_00000000001577_01,GAVIN VOLURE,30 rock,"WE'RE AT A PARTY, LEMON,\nTRY TO LOOSEN UP A L...",Television
3,8,8,GMO_00000000001585_01,FILE_MAF_20210727T231030Z_GMO_00000000001585_01,DO-OVER,30 rock,"[bossa nova music] ♪ LLO, PUSSYCAT! (male voic...",Television
4,9,9,GMO_00000000001598_01,FILE_MAF_20210727T231031Z_GMO_00000000001598_01,CUTBACKS,30 rock,"[cheers] >> WHOO!\n>> WHOO! >> OKAY, GUYS, I J...",Television


In [26]:
def getadword(episode, truthdatadf):
    for i in range(len(truthdatadf.lastRunJob)):
        if episode == truthdatadf.lastRunJob[i]:
            return truthdatadf.adword[i]

In [ ]:
# temp=targetlistcalc[27]
# temp=temp.replace('*News business use case needs to be scoped by NBCU - priority TBD', '')
# targetlistcalc[27]=temp

In [27]:
sampled_datadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,665,665,GMO_00000000040741_01,FILE_MAF_20210731T020535Z_GMO_00000000040741_01,I KNEW YOU WHEN,friday night lights,"LKS, IS IS SLAMMIN' SAMMY MEADE,\nFO [radio]\n...",Sports
1,8244,8244,GMO_00000000370441_01,FILE_MAF_20220215T090159Z_GMO_00000000370441_01,Make Up or Break Up?,total bellas,>> NIKKI: Yeah! Welcome to Paris! Tonight on t...,Sports
2,678,678,GMO_00000000039091_01,FILE_MAF_20220213T121447Z_GMO_00000000039091_01,HOMECOMING (EDITED),friday night lights,"What's after\nhigh school, Brian? Me and Mack ...",Sports
3,777,777,GMO_00000000039094_01,FILE_MAF_20220213T173625Z_GMO_00000000039094_01,NEVERMIND (EDITED),friday night lights,"Here's to God in ten years from now, Street, g...",Sports
4,679,679,GMO_00000000039095_01,FILE_MAF_20220213T173619Z_GMO_00000000039095_01,WHAT TO DO WHILE YOU'RE WAITING (EDITED),friday night lights,"Dad? You're home! Hello, Brian. Waverly? Damn,...",Sports


In [28]:
listofepisodes=sampled_datadf['lastRunJob'].to_list()

In [ ]:
#listofepisodes=['FILE_MAF_20220211T205601Z_GMO_00000000001313_01', 'FILE_MAF_20220211T205607Z_GMO_00000000001534_01', 'FILE_MAF_20210727T231031Z_GMO_00000000001577_01' ]

In [29]:
def keydict(dataframe, targetlist):
    d=defaultdict(list)
    for target in targetlist:
        minidf=dataframe.loc[dataframe['adword']==target]
        minilist=minidf['lastRunJob'].to_list()
        d[target]=minilist
    
    return d



In [30]:
targetkey_filelist=keydict(sampled_datadf, targetlist)


In [ ]:
# #NRS bulk with top 1 save to file regular
# dict_to_save=defaultdict(list)
# for target in tqdm(targetlist):
#     scores=[]
#     for ep in tqdm(listofepisodes):
#         index=materialID.index(ep)
#         groundtruth=getadword(ep,truthdatadf)
#         # if groundtruth in target_dict:
#         #         x=target_dict[groundtruth]
#         x=target_dict[target]
#         #print(groundtruth)
#         print(x)
#         segmentscore=[]
#         for i in range(len(newmetalist[index])):
#             if len(newmetalist[index][i])==0:
#                 continue
#             emb1=model.encode(newmetalist[index][i]) #, convert_to_tensor=True)
#             emb2=model.encode(targetlist) #, convert_to_tensor=True)
#             cos_sim=util.cos_sim(emb1,emb2)

#             #For OG Class
#             MaxAggregation = {}
#             for i in range(cos_sim.shape[1]):
#                 all_sentence_combinations=[]
#                 for j in range(cos_sim.shape[0]):
#                     all_sentence_combinations.append([cos_sim[j][i], j, i])
#                 all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
#                 MaxAggregation[i]=all_sentence_combinations[0]
                

#             segmentscore.append(MaxAggregation[x][0])

            
                    
#         avg=(sum(segmentscore)/len(segmentscore))
#         scores.append(avg)
        
#     print(scores)
#     dict_to_save[target]=scores

In [ ]:
# with open('NRS_top1_regular.pkl', 'wb') as f:
#     pickle.dump(dict_to_save, f)


In [ ]:
# with open('NRS_top1_regular.pkl', 'rb') as f:
#     NRS_dict=pickle.load(f)


In [31]:
def get_key(val):
    for key, value in target_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [32]:
dad=[]
for i in range(len(newnewmetalist)):
    mom=[]
    for j in range(len(newnewmetalist[i])):
        if len(newnewmetalist[i][j])==0:
            continue
        mom.append(newnewmetalist[i][j])
    dad.append(mom)

In [116]:
dad[0][0][0]

' just wanted to tell you that i appreciate you patching things up with him'

In [117]:
newnewmetalist[0][0][0]

' just wanted to tell you that i appreciate you patching things up with him'

In [33]:
#Relevance calculation per episode top1
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(dad[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(dad[index][l])==0: 
                continue
            emb1=model.encode(dad[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlistchild) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                MaxAggregation[k].append(all_sentence_combinations[0])
    
     
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
     

with open('Relevance_Top1_ChildAdword.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)   
 
        #RS and NRS for OG Class

        # RSlist=MaxAggregation[x]
        # score=[]
        # for s in RSlist:
        #     score.append(s[0])

        # rs=sum(score)/len(score)

        # NRSlist=NRS_dict[key]
        # nrsdenom=sum(NRSlist)-rs/len(NRSlist)

        # NRS=rs/nrsdenom

        
        # #RS and ClassPredicted for Predicted Class

        # order=[]
        # for key in MaxAggregation.keys():
        #     l=MaxAggregation[key]
        #     summ=[]
        #     for val in l:
        #         summ.append(val[0])
        #     order.append(sum(summ))
        
        
        # max_val=max(order)
        # max_index=order.index(max_val)
        # ClassPredicted=get_key(max_index)

        # RSlistss=MaxAggregation[max_index]
        # scoress=[]
        # for s in RSlistss:
        #     scoress.append(s[0])

        # RS_CP=sum(scoress)/len(scoress)

        # d={targetkey_filelist[key][i]:{'NRS': NRS, "Class_Predicted": ClassPredicted, "RS_CP": RS_CP, "RS_OG": rs }}

        


        


        

    

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
#Relevance calculation per episode top3
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(newmetalist[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(newmetalist[index][l])==0: 
                continue
            emb1=model.encode(newmetalist[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlist) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                templist=all_sentence_combinations[0:3]
                temp=[]
                sent=[]
                for item in templist:
                    temp.append(item[0])
                    sent.append(list([item[1],item[2]]))
                avg=(sum(temp)/len(temp))
                comb=list([avg, sent])
                MaxAggregation[k].append(comb)
            
            # print(MaxAggregation)
            
    
        # print(MaxAggregation)
        # print(dictofrelevance[key][i])
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
        # print(dictofrelevance[key][i])

with open('Relevance_AdditionalSignal_top3_regular.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)  

In [ ]:
#Relevance calculation per episode Harmonic
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(newmetalist[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(newmetalist[index][l])==0: 
                continue
            emb1=model.encode(newmetalist[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlist) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                all_sentence_combinations = [ele for ele in all_sentence_combinations if ele[0] > 0]
                templist=all_sentence_combinations
                temp=[]
                sent=[]
                for item in templist:
                    temp.append(float(item[0]))
                try:
                    sent.append([templist[0][1],templist[0][2],templist[0][0]])
                    avg=statistics.harmonic_mean(temp)
                    comb=list([avg, sent])
                    MaxAggregation[k].append(comb)
                except:
                    sent.append(['-'])
                    avg=0
                    comb=list([avg, sent])
                    MaxAggregation[k].append(comb)



            
            # print(MaxAggregation)
            
    
        # print(MaxAggregation)
        # print(dictofrelevance[key][i])
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
        # print(dictofrelevance[key][i])

with open('Relevance_Harmonic_regular.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)  

In [15]:
with open('Relevance_Top1_ChildAdword.pkl', 'rb') as f:
     Relevance_dict=pickle.load(f)

In [ ]:
print(Relevance_dict['Automotive'][0])

In [16]:
#Def Top1 Agg
def segmentcombiner(dic): #for segments to final dic of relevance scores
    dic1=copy.deepcopy(dic)
    filename=list(dic1.keys())[0]
    defdict=dic1[filename]
    for k,v in defdict.items():
        RSlist=defdict[k]
        score=[]
        for s in RSlist:
            score.append(s[0])

        rs=sum(score)/len(score)
        defdict[k]=rs
    d={filename:defdict}
    return d
    


In [ ]:
#Def Top3 and Harm Agg
# def segmentcombinertop3Harm(dic): #for segments to final dic of relevance scores
#     dic1=copy.deepcopy(dic)
#     filename=list(dic1.keys())[0]
#     defdict=dic1[filename]
#     for k,v in defdict.items():
#         RSlist=defdict[k][0]
#         score=[]
#         for s in RSlist:
#             score.append(s[0])

#         rs=sum(score)/len(score)
#         defdict[k]=rs
#     d={filename:defdict}
#     return d

In [ ]:
print(segmentcombiner(Relevance_dict_top3['Automotive'][18]))

In [ ]:
# with open('Relevance_regular.pkl', 'rb') as f:
#      Relevance_dict=pickle.load(f)

In [18]:
print(NRS(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': 1.1185518199181357, 'Books and Literature': 1.1806155004121979, 'Business and Finance': 1.333218089144164, 'Education': 1.1651432565277338, 'Events and Attractions': 1.1272685094122596, 'Family and Relationships': 1.0931180301453156, 'Food & Drink': 1.6569397836646746, 'Hobbies & Interests': 1.0367267053895628, 'Medical Health': 1.6891036298829092, 'Music and Audio': 1.0987961579827805, 'Pets': 0.9013374717644428, 'Real Estate': 2.84922916149329, 'Religion & Spirituality': 1.2678996187919302, 'Science': 1.1590756019772757, 'Shopping': 1.1895382431765187, 'Sports': 1.1372730819655776, 'Style & Fashion': 1.6912651546258568, 'Technology & Computing': 1.7057414794374346, 'Television': 1.339820876652496, 'Travel': 1.3563558891567618}
None


In [17]:
#Def NRS for top1 
def NRS(dict):
    d={}
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for target in tqdm(targetlist): #for each topic
            scoreforfiles=[]
            if len(dict[target]) == 0:
                continue
            for i in range(len(dict[k])): #for each file in label
                x=target_dict[target]
                combinedsegments=segmentcombiner(dict[k][i])
                key=list(combinedsegments.keys())[0]
                scoreforfiles.append(float(combinedsegments[key][x])) #combine segments 
            avg=sum(scoreforfiles)/len(scoreforfiles)
            d1[target]=avg
        d[k]=d1

    finaldict={}
    for k,v in d.items():
        numerator=d[k][k]
        denominator=[]
        for kk,vv in d.items():
            denominator.append(d[kk][k])
        avgdenominator=(sum(denominator)-numerator)/(len(denominator)-1)
        NRS=numerator/avgdenominator
        finaldict[k]=NRS
    print(finaldict)


            
        

In [ ]:
print(RSMetrics(Relevance_dict))

In [ ]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetrics(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            scoreforfiles.append(round(float(combinedsegments[key][x]),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        
    


In [ ]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetricsHM(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            ll=(dict[k][i][key][x])
            lll=[]
            for score in ll:
                print(score[1])
                lll.append(score[1][0][2])
            avg=sum(lll)/len(ll)
            scoreforfiles.append(round(float(avg),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        

In [ ]:
print(Prediction(Relevance_dict))

In [19]:
#Def Predicted Class and RS of Class
def Prediction(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for i in range(len(dict[k])): #for each file in label
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            order=[]
            for kk in combinedsegments[key].keys():
                l=combinedsegments[key][kk]
                order.append(l)
            max_val=max(order)
            max_index=order.index(max_val)
            ClassPredicted=get_key(max_index)
            RS_CP=combinedsegments[key][max_index]
            d2={'ClassPredicted': ClassPredicted, 'RS_CP': RS_CP }
            d1[key]=d2
        d[k]=d1

    return d

            # 
            # scoreforfiles.append(float(combinedsegments[key][x]0))

            # order=[]
            # for key in MaxAggregation.keys():
            #     l=MaxAggregation[key]
            #     summ=[]
            #     for val in l:
            #         summ.append(val[0])
            #     order.append(sum(summ))
            
            
            # max_val=max(order)
            # max_index=order.index(max_val)
            # ClassPredicted=get_key(max_index)

            # RSlistss=MaxAggregation[max_index]
            # scoress=[]
            # for s in RSlistss:
            #     scoress.append(s[0])

            # RS_CP=sum(scoress)/len(scoress)
    

In [ ]:
#Def Predicted Class and RS of Class HM
def PredictionHM(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for i in range(len(dict[k])): #for each file in label
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            order=[]
            for kk in combinedsegments[key].keys():
                l=combinedsegments[key][kk]
                order.append(l)
            max_val=max(order)
            max_index=order.index(max_val)
            ClassPredicted=get_key(max_index)

            ll=(dict[k][i][key][max_index])
            lll=[]
            for score in ll:
                lll.append(score[1][0][2])
            avg=sum(lll)/len(ll)


            RS_CP=float(avg)
            d2={'ClassPredicted': ClassPredicted, 'RS_CP': RS_CP }
            d1[key]=d2
        d[k]=d1

    return d

            # 
            # scoreforfiles.append(float(combinedsegments[key][x]0))

            # order=[]
            # for key in MaxAggregation.keys():
            #     l=MaxAggregation[key]
            #     summ=[]
            #     for val in l:
            #         summ.append(val[0])
            #     order.append(sum(summ))
            
            
            # max_val=max(order)
            # max_index=order.index(max_val)
            # ClassPredicted=get_key(max_index)

            # RSlistss=MaxAggregation[max_index]
            # scoress=[]
            # for s in RSlistss:
            #     scoress.append(s[0])

            # RS_CP=sum(scoress)/len(scoress)
    

In [21]:
print(mAP(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

({'Automotive': 5.0, 'Books and Literature': 5.0, 'Business and Finance': 10.0, 'Education': 0.0, 'Events and Attractions': 40.0, 'Family and Relationships': 10.0, 'Food & Drink': 65.0, 'Hobbies & Interests': 0.0, 'Medical Health': 70.0, 'Music and Audio': 60.0, 'Pets': 0.0, 'Real Estate': 85.0, 'Religion & Spirituality': 0.0, 'Science': 0.0, 'Shopping': 5.0, 'Sports': 0.0, 'Style & Fashion': 100.0, 'Technology & Computing': 10.0, 'Television': 5.0, 'Travel': 10.0}, 24.0)


In [20]:
#Def mAP
def mAP(dic):
    res=Prediction(dic)
    d={}
    mAP=[]
    for k,v in res.items():
        classes=[]
        truth=[]

        for kk,vv in res[k].items():
            ClassPred=res[k][kk]['ClassPredicted']
            if ClassPred==k:
                classes.append(ClassPred)
            truth.append(k)
        precision=(len(classes)/len(truth))*100
        mAP.append(precision)
        d[k]=precision
    mAPfinal=sum(mAP)/len(mAP)
    return(d,mAPfinal)
  




In [ ]:
Relevance_dict['Automotive'][0]

In [138]:
def topksentences(dict):
    d=defaultdict(list)
    adwords=[]
    texts=[]
    scores=[]
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
 
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            seg_rel=copy.deepcopy(dict[k][i])
            filename=list(seg_rel.keys())[0]
            defdict=seg_rel[filename]
            segments=defdict[x]
            # for count,seg1 in enumerate(segments):
            #     if len(seg1)==0:
            #         segments[count].append('Buffer')


            for count,seg in enumerate(segments):
                seg.append(filename)
                seg.append(count)                
                scoreforfiles.append(seg)
        sort=sorted(scoreforfiles, key=lambda x: x[0], reverse=True)
        Final=sort[0:20]
        # print(Final)
        
        for items in Final:
            index=materialID.index(items[3])
            try:
                items.append(dad[index][items[4]][items[1]])
            except Exception as e:
                print(e)
                print(index,items[4],items[1])
                # print(dad[index][items[4]][items[1]])
                print(newnewmetalist[index][items[4]][items[1]])
                # items.append(dad[index][items[4]][items[1]])
                continue
        # print(Final)
        ff=[]
        for f in Final:
            dic={}
            temp=str(f[0])
            temp=temp.replace('tensor(', "")
            temp=temp.replace(')', "")
            dic['Adword']=k
            dic['Text']=f[5]
            dic['Score']=temp
            
            fff.append(dic)
    
        # d[k].extend(ff)
    return fff
    
        

                
                

In [ ]:
print(materialID.index('FILE_MAF_20210729T074455Z_GMO_00000000044309_01'))

In [139]:
dict_top20=topksentences(Relevance_dict)

  0%|          | 0/29 [00:00<?, ?it/s]

In [140]:
df11=pd.DataFrame.from_list(dict_top20)

AttributeError: type object 'DataFrame' has no attribute 'from_list'

In [137]:
df11.head()

,Automotive,Books and Literature,Business and Finance,Education,Events and Attractions,Family and Relationships,Food & Drink,Hobbies & Interests,Medical Health,Music and Audio,Pets,Real Estate,Religion & Spirituality,Science,Shopping,Sports,Style & Fashion,Technology & Computing,Television,Travel
0,"{'Adword': 'Automotive', 'Text': 'the most suc...","{'Adword': 'Books and Literature', 'Text': 'fl...","{'Adword': 'Business and Finance', 'Text': 'an...","{'Adword': 'Education', 'Text': 'the grades th...","{'Adword': 'Events and Attractions', 'Text': '...","{'Adword': 'Family and Relationships', 'Text':...","{'Adword': 'Food & Drink', 'Text': 'friday's e...","{'Adword': 'Hobbies & Interests', 'Text': 'any...","{'Adword': 'Medical Health', 'Text': 'there ar...","{'Adword': 'Music and Audio', 'Text': 'anythin...","{'Adword': 'Pets', 'Text': 'my spare set of ca...","{'Adword': 'Real Estate', 'Text': ' as the ma...","{'Adword': 'Religion & Spirituality', 'Text': ...","{'Adword': 'Science', 'Text': 'chandra the wor...","{'Adword': 'Shopping', 'Text': ' anything in t...","{'Adword': 'Sports', 'Text': 'talk about a who...","{'Adword': 'Style & Fashion', 'Text': ''s"" fas...","{'Adword': 'Technology & Computing', 'Text': '...","{'Adword': 'Television', 'Text': 'if i know an...","{'Adword': 'Travel', 'Text': ' itinerary reque..."
1,"{'Adword': 'Automotive', 'Text': 'cb radio sta...","{'Adword': 'Books and Literature', 'Text': ' m...","{'Adword': 'Business and Finance', 'Text': 'uh...","{'Adword': 'Education', 'Text': ' why don't yo...","{'Adword': 'Events and Attractions', 'Text': '...","{'Adword': 'Family and Relationships', 'Text':...","{'Adword': 'Food & Drink', 'Text': 'different ...","{'Adword': 'Hobbies & Interests', 'Text': ' m...","{'Adword': 'Medical Health', 'Text': ' i want ...","{'Adword': 'Music and Audio', 'Text': 'camera ...","{'Adword': 'Pets', 'Text': 'oh the new tiffany...","{'Adword': 'Real Estate', 'Text': ' and real ...","{'Adword': 'Religion & Spirituality', 'Text': ...","{'Adword': 'Science', 'Text': 'i know a scient...","{'Adword': 'Shopping', 'Text': ' what do you ...","{'Adword': 'Sports', 'Text': ' what some next...","{'Adword': 'Style & Fashion', 'Text': ' moans ...","{'Adword': 'Technology & Computing', 'Text': '...","{'Adword': 'Television', 'Text': ' so much of ...","{'Adword': 'Travel', 'Text': ' bucket list ch..."
2,"{'Adword': 'Automotive', 'Text': 'it'll have a...","{'Adword': 'Books and Literature', 'Text': ' d...","{'Adword': 'Business and Finance', 'Text': 'i ...","{'Adword': 'Education', 'Text': ' there's boar...","{'Adword': 'Events and Attractions', 'Text': '...","{'Adword': 'Family and Relationships', 'Text':...","{'Adword': 'Food & Drink', 'Text': 'different ...","{'Adword': 'Hobbies & Interests', 'Text': ' so...","{'Adword': 'Medical Health', 'Text': ' your me...","{'Adword': 'Music and Audio', 'Text': ' sighs...","{'Adword': 'Pets', 'Text': 'and the furnishing...","{'Adword': 'Real Estate', 'Text': 'we just got...","{'Adword': 'Religion & Spirituality', 'Text': ...","{'Adword': 'Science', 'Text': 'i'm sort of int...","{'Adword': 'Shopping', 'Text': ' sighs what ...","{'Adword': 'Sports', 'Text': 'man we are athle...","{'Adword': 'Style & Fashion', 'Text': 'at stak...","{'Adword': 'Technology & Computing', 'Text': '...","{'Adword': 'Television', 'Text': 'the phone in...","{'Adword': 'Travel', 'Text': ' if you find som..."
3,"{'Adword': 'Automotive', 'Text': ' engine 51 t...","{'Adword': 'Books and Literature', 'Text': 'i ...","{'Adword': 'Business and Finance', 'Text': 'wo...","{'Adword': 'Education', 'Text': ' it's your sc...","{'Adword': 'Events and Attractions', 'Text': '...","{'Adword': 'Family and Relationships', 'Text':...","{'Adword': 'Food & Drink', 'Text': 'you must c...","{'Adword': 'Hobbies & Interests', 'Text': 'the...","{'Adword': 'Medical Health', 'Text': ' you peo...","{'Adword': 'Music and Audio', 'Text': 'rock mu...","{'Adword': 'Pets', 'Text': 'i've got some shop...","{'Adword'

In [ ]:
df11.to_csv("Top20Sentences.csv")